In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

#alwaysimport this for every pyspark analytics
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date
from pyspark.sql.functions import from_utc_timestamp
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


import ts.flint
from ts.flint import FlintContext
flintContext = FlintContext(spark)

In [2]:

from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import mean, col


sp500 = spark.read.option('header', True).option('inferSchema', True).csv("C:\\Users\\brono\\OneDrive - NetGuardians SA\\Desktop2\\023-Coop\\Test_last5.csv").withColumnRenamed('RESPONSETIME_CHANNEL_parsed', 'time')
#sp500.show(100)
 

sp500 = sp500.withColumn('timenew',sp500.time.astype('Timestamp').cast("long")).sort(col('timenew'))

sp500 = sp500.withColumn('partition',F.lit("1234567"))

#sp500.show()

sp500x =sp500.filter(sp500.NARRATION.contains('SUCCESS'))
sp500x1 =sp500x.filter(sp500.timenew < 1581665845)



w = Window.partitionBy('partition').orderBy('timenew').rangeBetween(-60*60,0)
sp500x2 = sp500x1.withColumn('occurrences_in_5_min',F.count('timenew').over(w)).withColumn('sumin30mins',F.sum('FIELD4').over(w))
w21 = Window.partitionBy('partition')


#.orderBy('timenew')
sp500x20 = sp500x2.withColumn('maxcount',F.max('occurrences_in_5_min').over(w21)).withColumn('maxsum',F.max('sumin30mins').over(w21))
sp500xfin =sp500x20.select(col('partition').alias('partition2'),'maxsum','maxcount')
sp500xfin =sp500xfin.dropDuplicates(subset=['partition2'])

sp500xfin.show()
#sp500x2.sort(desc("occurrences_in_5_min"),desc("sumin30mins"))
#.show()

#maxcount, max_sum = sp500x2.select(max("occurrences_in_5_min"), max("sumin30mins")).first()




sp500x3 =sp500x.filter(sp500.timenew > 1581665845)

w = Window.partitionBy('partition').orderBy('timenew').rangeBetween(-60*60,0)
sp500x3 = sp500x3.withColumn('occurrences_in_5_min',F.count('timenew').over(w)).withColumn('sumin30mins',F.sum('FIELD4').over(w))



#sp500x3.show()

finaldf = sp500x3.join(sp500xfin, sp500x3.partition == sp500xfin.partition2, how='cross')

#finaldf = finaldf.dropDuplicates(subset=['timenew','FIELD4']).filter((finaldf.occurrences_in_5_min > finaldf.maxcount) & (finaldf.sumin30mins > finaldf.maxsum))

#finaldf.show()


finaldf = finaldf.filter((finaldf.occurrences_in_5_min > finaldf.maxcount) & (finaldf.sumin30mins > finaldf.maxsum))


finaldf.sort(finaldf.timenew).show()



#sp500x4 =sp500x3.filter((sp500x3.occurrences_in_5_min > maxcount) & (sp500x3.sumin30mins > max_sum))

#maxdate, min_date = sp500x4.select(max("timenew"), min("timenew")).first()

#lowerdate=maxdate+-60*45*12

#sp500x4.show(100)


#w3 = Window.partitionBy('FIELD102').orderBy('timenew').rangeBetween(-60*30,maxdate)
#sp500final = sp500x.filter(F.col("timenew").between(lowerdate,maxdate))


#sp500final.show(100)


#sp5001=sp5001.filter(sp500.DATEX_parsed.contains('2020-02-14'))




#limit = (sp5001.select(F.max(F.col("occurrences_in_5_min"))).limit(1).collect()[0])




#print(maxdate)

#print(lowerdate)
#print(maxcount)
#print(max_sum)
#desc("recent"), asc("Freq"), asc("Monitor")




+----------+------+--------+
|partition2|maxsum|maxcount|
+----------+------+--------+
|   1234567|130000|       3|
+----------+------+--------+

+-------------+-------------+-----+-------------------+------+-------+---------+--------------------+--------------------+----------+---------+--------------------+-----------+----------+------+--------+
|     FIELD102|     FIELD103|DATEX|       DATEX_parsed|FIELD4|FIELD39|NARRATION|             FIELD68|                time|   timenew|partition|occurrences_in_5_min|sumin30mins|partition2|maxsum|maxcount|
+-------------+-------------+-----+-------------------+------+-------+---------+--------------------+--------------------+----------+---------+--------------------+-----------+----------+------+--------+
| 254720489010|         null|43875|2020-02-14 03:00:00| 45100|      0|  SUCCESS|254720489010 2547...|2020-02-14 11:07:...|1581667631|  1234567|                   4|     615100|   1234567|130000|       3|
| 254720489010|1240000000000|43875|202

In [3]:
sp500x4.printSchema()

NameError: name 'sp500x4' is not defined

In [4]:
sp50013 =sp500x3.filter(sp500x3.DATEX_parsed.contains('2020-02-14'))
sp50013.sort(col('timenew')).show(100)

+-------------+-------------+-----+-------------------+------+-------+---------+--------------------+--------------------+----------+---------+--------------------+-----------+
|     FIELD102|     FIELD103|DATEX|       DATEX_parsed|FIELD4|FIELD39|NARRATION|             FIELD68|                time|   timenew|partition|occurrences_in_5_min|sumin30mins|
+-------------+-------------+-----+-------------------+------+-------+---------+--------------------+--------------------+----------+---------+--------------------+-----------+
|1109039870300| 254724000000|43875|2020-02-14 03:00:00| 35000|      0|  SUCCESS|REV-0110903987030...|2020-02-14 10:37:...|1581665846|  1234567|                   1|      35000|
|1109039870300|         null|43875|2020-02-14 03:00:00| 35000|      0|  SUCCESS|01109039870300 25...|2020-02-14 10:46:...|1581666413|  1234567|                   3|     570000|
|1109039870300|         null|43875|2020-02-14 03:00:00|500000|      0|  SUCCESS|979326 KES 500000...|2020-02-14 10:

In [5]:
#sp500 = sp500.withColumn('counter',
 # F.when((F.col("occurrences_in_5_min") > 4), "True")\
#.otherwise("False")
#)

#sp5004 =sp500.filter(sp500.occurrences_in_5_min > 1)
sp5004 =sp5001.filter(sp500.DATEX_parsed.contains('2018-04-26'))
sp5004.sort(col('timenew')).show(100)

NameError: name 'sp5001' is not defined

In [ ]:
#from pyspark.sql.window import Window
#from pyspark.sql.functions import *
#import numpy as np 
#from pyspark.sql.types import FloatType

#w = Window.partitionBy('FIELD102').orderBy('timenew').rangeBetween(-60*30,0)
#count_udf = udf(lambda x: float(np.count(x)), FloatType())

#sp5001.filter(sp500.DATEX_parsed.contains('2020-02-14')).withColumn("list", collect_list("timenew").over(w)) \
 # .withColumn("rolling_count", count_udf("list")).show(truncate = False)

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import numpy as np 
from pyspark.sql.types import FloatType


w = Window.partitionBy('FIELD102').orderBy('timenew').rangeBetween(-60*30,0)
count_udf = udf(lambda x: float(count(x)), FloatType())

#sp50067=sp5001.filter(sp500.DATEX_parsed.contains('2020-02-14'))
sp50067=sp5001.withColumn("list", collect_list("timenew").over(w))

sp50067=sp50067.withColumn("rolling_count",size(sp50067.list))

sp50067=sp50067.filter(sp500.DATEX_parsed.contains('2020-02-14'))

sp50067.sort(col('timenew')).show(100)

sp5004.sort(col('timenew')).show(100)

NameError: name 'sp5001' is not defined